In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import xgboost as xgb


In [80]:
df = pd.read_csv('SolAtasIMC_tratado.csv')
df.head()

,date,open,high,low,close,value
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25,75
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15,75
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30,75
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30,75
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25,75


In [81]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36400 entries, 0 to 36399
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    36400 non-null  object 
 1   open    36400 non-null  float64
 2   high    36400 non-null  float64
 3   low     36400 non-null  float64
 4   close   36400 non-null  float64
 5   value   36400 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 1.7+ MB
None


# Preprocesado de Datos

In [82]:
tamanio = df.shape[0]

In [83]:
df_train = df.copy().loc[0:int(tamanio*0.7)]
df_train

,date,open,high,low,close,value
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25,75
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15,75
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30,75
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30,75
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25,75
...,...,...,...,...,...,...
25476,2023-07-11 21:00:00,22.00,22.05,21.90,22.00,57
25477,2023-07-11 22:00:00,22.00,22.10,21.90,22.00,57
25478,2023-07-11 23:00:00,22.00,22.05,21.75,21.95,57
25479,2023-07-12 00:00:00,21.95,22.10,21.90,22.05,64


In [84]:
df_vali = df.copy().loc[int(tamanio*0.7 + 1):int(tamanio*0.9)]
df_vali

,date,open,high,low,close,value
25481,2023-07-12 02:00:00,22.10,22.30,22.05,22.15,64
25482,2023-07-12 03:00:00,22.15,22.25,22.10,22.10,64
25483,2023-07-12 04:00:00,22.10,22.10,22.00,22.00,64
25484,2023-07-12 05:00:00,22.00,22.00,21.90,21.95,64
25485,2023-07-12 06:00:00,21.95,22.05,21.90,22.00,64
...,...,...,...,...,...,...
32756,2024-05-10 05:00:00,153.65,154.35,152.85,153.95,66
32757,2024-05-10 06:00:00,153.95,154.70,153.45,153.75,66
32758,2024-05-10 07:00:00,153.75,154.10,152.30,153.30,66
32759,2024-05-10 08:00:00,153.30,155.10,153.15,154.95,66


In [85]:
df_test = df.copy().loc[int(tamanio*0.9 + 1):tamanio]
df_test

,date,open,high,low,close,value
32761,2024-05-10 10:00:00,154.35,154.5,153.45,154.10,66
32762,2024-05-10 11:00:00,154.10,154.8,153.25,154.15,66
32763,2024-05-10 12:00:00,154.15,154.3,153.25,154.15,66
32764,2024-05-10 13:00:00,154.15,155.2,153.00,155.05,66
32765,2024-05-10 14:00:00,155.05,155.4,153.10,153.30,66
...,...,...,...,...,...,...
36395,2024-10-08 20:00:00,143.35,143.9,142.35,142.95,49
36396,2024-10-08 21:00:00,142.95,144.1,142.25,143.75,49
36397,2024-10-08 22:00:00,143.75,144.5,143.35,144.50,49
36398,2024-10-08 23:00:00,144.50,144.7,144.05,144.25,49


In [86]:
df_valitest = pd.concat([df_vali, df_test], axis=0)

Numero de horas que se utilizan en la predicción

In [87]:
numhorasconst = 4

# Predicción Utilizando XGBoost

In [88]:
def create_df_n(df, n):
    df_aux = df.copy()
    for i in range(1, n):
        df_aux['open_before' + str(i)] = df_aux['open'].shift(+i)
        df_aux['high_before' + str(i)] = df_aux['high'].shift(+i)
        df_aux['low_before' + str(i)] = df_aux['low'].shift(+i)
        df_aux['close_before' + str(i)] = df_aux['close'].shift(+i)
        df_aux['value_before' + str(i)] = df_aux['value'].shift(+i)
    df_aux['close_next'] = df_aux['close'].shift(-1)
    df_aux = df_aux.dropna()
    return df_aux

Ponemos los datos en el formato correcto

In [89]:
df_xgb = create_df_n(df, 1)


In [90]:
tamanio_xgb = df_xgb.shape[0]
tamanio_xgb

36399

In [91]:
def createdftrain(df_aux):
    tamanio_aux = df_aux.shape[0]
    return df_aux.copy().iloc[0:int(tamanio_aux*0.7)]

In [92]:
def createdfvali(df_aux):
    tamanio_aux = df_aux.shape[0]
    return df_aux.copy().iloc[int(tamanio_aux*0.7 + 1):int(tamanio_aux*0.9)]

In [93]:
def createdftest(df_aux):
    tamanio_aux = df_aux.shape[0]
    return df_aux.copy().iloc[int(tamanio_aux*0.9 + 1):tamanio_aux]

In [94]:
def preparar_datosXGBoost(df_aux):
    X = df_aux.drop(['date', 'close_next'], axis=1)
    #X = df_aux[['open', 'high', 'low', 'close', 'open_before1', 'high_before1', 'low_before1', 'close_before1']] 
    y = df_aux['close_next']
    return (xgb.DMatrix(data=X, label=y), y)

In [95]:
df_train_xgb = createdftrain(df_xgb)
df_train_xgb

,date,open,high,low,close,value,close_next
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25,75,3.15
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15,75,3.30
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30,75,3.30
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30,75,3.25
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25,75,3.25
...,...,...,...,...,...,...,...
25474,2023-07-11 19:00:00,22.10,22.20,22.00,22.10,57,22.00
25475,2023-07-11 20:00:00,22.10,22.10,21.95,22.00,57,22.00
25476,2023-07-11 21:00:00,22.00,22.05,21.90,22.00,57,22.00
25477,2023-07-11 22:00:00,22.00,22.10,21.90,22.00,57,21.95


In [96]:
df_vali_xgb = createdfvali(df_xgb)
df_vali_xgb

,date,open,high,low,close,value,close_next
25480,2023-07-12 01:00:00,22.05,22.15,22.00,22.10,64,22.15
25481,2023-07-12 02:00:00,22.10,22.30,22.05,22.15,64,22.10
25482,2023-07-12 03:00:00,22.15,22.25,22.10,22.10,64,22.00
25483,2023-07-12 04:00:00,22.10,22.10,22.00,22.00,64,21.95
25484,2023-07-12 05:00:00,22.00,22.00,21.90,21.95,64,22.00
...,...,...,...,...,...,...,...
32754,2024-05-10 03:00:00,152.05,155.00,151.45,153.40,66,153.65
32755,2024-05-10 04:00:00,153.40,153.80,152.70,153.65,66,153.95
32756,2024-05-10 05:00:00,153.65,154.35,152.85,153.95,66,153.75
32757,2024-05-10 06:00:00,153.95,154.70,153.45,153.75,66,153.30


In [97]:
df_test_xgb = createdftest(df_xgb)
df_test_xgb

,date,open,high,low,close,value,close_next
32760,2024-05-10 09:00:00,154.95,155.75,154.25,154.35,66,154.10
32761,2024-05-10 10:00:00,154.35,154.50,153.45,154.10,66,154.15
32762,2024-05-10 11:00:00,154.10,154.80,153.25,154.15,66,154.15
32763,2024-05-10 12:00:00,154.15,154.30,153.25,154.15,66,155.05
32764,2024-05-10 13:00:00,154.15,155.20,153.00,155.05,66,153.30
...,...,...,...,...,...,...,...
36394,2024-10-08 19:00:00,143.80,143.95,142.40,143.35,49,142.95
36395,2024-10-08 20:00:00,143.35,143.90,142.35,142.95,49,143.75
36396,2024-10-08 21:00:00,142.95,144.10,142.25,143.75,49,144.50
36397,2024-10-08 22:00:00,143.75,144.50,143.35,144.50,49,144.25


In [98]:
df_valitest_xgb = pd.concat([df_vali_xgb, df_test_xgb], ignore_index=True)

In [99]:
XvtXGB = df_valitest_xgb[['open', 'high', 'low', 'close', 'value']] 
yvtXGB = df_valitest_xgb['close_next']

In [100]:
dtrainvali = xgb.DMatrix(data=XvtXGB, label=yvtXGB)

Definimos las características para el entrenamiento

Comprobación de la tasa de error en los datos de test

In [101]:
def evalXGB(Test_xgb, predict_xgb_test):
    suma = 0
    n = len(Test_xgb)
    for i in range(0,n):
        suma = abs(predict_xgb_test[i] - Test_xgb[i])/Test_xgb[i] +  suma
    error_medio = suma/n
    emp = error_medio*100 # error medio en porcentaje
    return emp

In [102]:
def train_XGB_depth(maxdepthmin, maxdepthmax, dtrainf, dvalif, dtestf, ytest):
    resultados = []
    best = 100
    best_depth = 0
    for i in range(maxdepthmin, maxdepthmax):
        param = {'max_depth': i, 'eta': 0.1, 'objective': 'reg:squarederror'}
        evals = [(dtrainf, 'train'), (dvalif, 'validacion')]
        bstaux = xgb.train(param, dtrainf, num_boost_round=1000, evals=evals, early_stopping_rounds=10, verbose_eval=10)
        predict_xgb_test = bstaux.predict(dtestf)
        valor = evalXGB(ytest, predict_xgb_test)
        cadena = "Modelos/modelo_xgb_v.json" + str(valor)+ "_d" + str(i) + "_eta" + str(param.get("eta")) + ".json"
        bstaux.save_model(cadena) 
        resultados.append({'max_depth': i, 'eta': param['eta'], 'valor': valor})
        if(valor < best):
            best = valor
            best_depth = i
    return (best_depth, best, resultados)

In [103]:
def trainGlobalXGB(inid, find, inih, finh):
    best = 100
    best_depth = 0
    for i in range(inih, finh):
        df_aux = create_df_n(df, i)
        dtrain_aux = createdftrain(df_aux)
        dvali_aux = createdfvali(df_aux)
        dtest_aux = createdftest(df_aux)
        dtrain_prep = preparar_datosXGBoost(dtrain_aux)
        dvali_prep = preparar_datosXGBoost(dvali_aux)
        dtest_prep = preparar_datosXGBoost(dtest_aux)
        values = train_XGB_depth(inid, find, dtrain_prep[0], dvali_prep[0], dtest_prep[0], dtest_prep[1].values)
        print(str(i)+" "+str(values[0])+" "+str(values))
        df_resultados = pd.DataFrame(values[2])
        cadena = "Dataframes/resultados_xgboost_h" + str(i) + ".csv"
        df_resultados.to_csv(cadena, index=False)
        with open('OptimizaciónXGBoostIMC.txt', 'a') as archivo:
            archivo.write("Número de horas: "+str(i)+" Profundidad: "+str(values[0])+" Valor de emp obtenido: "+str(values[1]) + "\n")
        if(values[1] < best):
            best = values[1]
            best_depth = values[0]
    return best, best_depth

In [ ]:
data = trainGlobalXGB(1, 151, 1, 169)

[0]	train-rmse:53.27032	validacion-rmse:55.23101
[10]	train-rmse:26.63456	validacion-rmse:21.66451
[20]	train-rmse:15.80969	validacion-rmse:9.74188
[30]	train-rmse:10.10780	validacion-rmse:6.02804
[40]	train-rmse:6.72884	validacion-rmse:4.32847
[50]	train-rmse:4.62899	validacion-rmse:3.40634
[60]	train-rmse:3.34703	validacion-rmse:2.84333
[70]	train-rmse:2.60043	validacion-rmse:2.59538
[80]	train-rmse:2.18532	validacion-rmse:2.50216
[90]	train-rmse:1.96380	validacion-rmse:2.44815
[100]	train-rmse:1.84541	validacion-rmse:2.42618
[110]	train-rmse:1.78429	validacion-rmse:2.41209
[120]	train-rmse:1.75331	validacion-rmse:2.40979
[130]	train-rmse:1.73741	validacion-rmse:2.40922
[140]	train-rmse:1.72917	validacion-rmse:2.40845
[150]	train-rmse:1.72466	validacion-rmse:2.40665
[160]	train-rmse:1.72175	validacion-rmse:2.40448
[170]	train-rmse:1.71906	validacion-rmse:2.39564
[180]	train-rmse:1.71645	validacion-rmse:2.38706
[190]	train-rmse:1.71393	validacion-rmse:2.37956
[200]	train-rmse:1.71149	